We will try to scrape salary data for each player listed in the Historical_RaptorData.csv 


In [1]:
import pandas as pd
import numpy as np

In [3]:
histor_raptor_player = pd.read_csv("../Data/historical_RAPTOR_by_player.csv")

In [4]:
histor_raptor_player

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
0,Alaa Abdelnaby,abdelal01,1991,640,303,-3.938450,-0.510076,-4.448526,-0.265191,-0.226163,-0.039028,-3.905732,-1.696796,-5.602528,0.199241
1,Alaa Abdelnaby,abdelal01,1992,1998,959,-2.553849,-0.197943,-2.751792,-0.006893,0.070322,-0.077215,-2.499576,-0.721876,-3.221452,0.485436
2,Alaa Abdelnaby,abdelal01,1993,2754,1379,-2.373736,-2.069808,-4.443544,-1.227189,-0.845272,-0.381918,-2.328144,-2.002263,-4.330407,0.488266
3,Alaa Abdelnaby,abdelal01,1994,320,159,-6.140056,-2.748312,-8.888368,-0.500600,-0.500600,0.000000,-5.820936,-3.526560,-9.347496,0.511152
4,Alaa Abdelnaby,abdelal01,1995,984,506,-3.846543,-1.268012,-5.114556,-0.615802,-0.615802,0.000000,-3.715429,-1.027024,-4.742453,1.545922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18548,Ivica Zubac,zubaciv01,2017,1305,609,-3.358178,-3.040589,-6.398767,-1.141160,-1.141160,0.000000,-3.533604,-2.932103,-6.465707,0.644128
18549,Ivica Zubac,zubaciv01,2018,871,410,-2.903709,2.688832,-0.214877,0.531968,0.531968,0.000000,-2.616331,0.752875,-1.863456,0.311576
18550,Ivica Zubac,zubaciv01,2019,2345,1079,-2.362444,1.813768,-0.548676,1.217134,1.114995,0.102138,-2.810283,1.157824,-1.652459,0.077074
18551,Ivica Zubac,zubaciv01,2020,3447,1646,1.099849,3.549458,4.649308,6.224207,5.020903,1.203304,-0.310842,3.331974,3.021132,-0.410164


### Example of how salary data is shown:
---
It is present as a table under the page for each player as shown [here](https://www.basketball-reference.com/players/j/jamesle01.html#all_salaries).
The URL for LeBron James is:
```
https://www.basketball-reference.com/players/j/jamesle01.html#all_salaries
```
<!-- <script src="https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2Fj%2Fjamesle01.html&div=div_all_salaries"></script> -->

However, we can't access the table directly since it's loaded in from a SQL backend and HTML is used to wrap the table but doesn't show the values directly. Instead, we have to use the url of the widget for the table:

```
https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2Fj%2Fjamesle01.html&div=div_all_salaries
```

We can replace `%2Fplayers%2Fj%2Fjamesle01` with `%2Fplayers%2Fa%2Fabdelal01` to get the url for NBA player Alaa Abdelnaby:

```
https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2Fa%2Fabdelal01.html&div=div_all_salaries
```

In other words, the link for each player's salary data is essentially deliminated as:

`https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2F` + `<letter_of_last_name>` + `%2F` + `<player_id>` + `.html&div=div_all_salaries`
    
The `player_id` is readily available as a column in the `histor_raptor_player`.

### Scrape 1 page:
---
https://medium.com/analytics-vidhya/intro-to-scraping-basketball-reference-data-8adcaa79664a

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
url = "https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2Fj%2Fjamesle01.html&div=div_all_salaries"
# get HTML data
html = urlopen(url)

# get bs4 obj from HTML
soup = BeautifulSoup(html, features = "lxml")

Each row stores the season as a `th` while all other attributes can be accessed as `td`.

In [6]:
soup.find_all("tr", limit = 1)[0]
# each row stores the season as a th while all other attributes can be accessed as 

<tr><th aria-label="Season" class="poptip sort_default_asc center" data-stat="season" data-tip="If listed as single number, the year the season ended.&lt;br&gt;★ - Indicates All-Star for league.&lt;br&gt;Only on regular season tables." scope="col">Season</th><th aria-label="Team" class="poptip sort_default_asc left" data-stat="team_name" scope="col">Team</th><th aria-label="Lg" class="poptip sort_default_asc center" data-stat="lg_id" data-tip="League" scope="col">Lg</th><th aria-label="Salary" class="poptip right" data-stat="salary" scope="col">Salary</th></tr>

In [7]:
# get all headers of table as a list
headers = [th.getText() for th in soup.findAll('tr', limit=1)[0].findAll('th')]
print(headers)
# get all headers of table as a list
rows = soup.findAll('tr')[1:]
rows_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]

# change money to numeric
for row in rows_data:
    row[-1] = int(row[-1].strip('$').replace(',',''))
    
df = pd.DataFrame(rows_data)
df.columns = headers[1:]
df[headers[0]] = [[td.getText() for td in rows[i].findAll('th')][0] for i in range(len(rows))]
df = df[headers]
df

['Season', 'Team', 'Lg', 'Salary']


,Season,Team,Lg,Salary
0,2003-04,Cleveland Cavaliers,NBA,4018920
1,2004-05,Cleveland Cavaliers,NBA,4320360
2,2005-06,Cleveland Cavaliers,NBA,4621800
3,2006-07,Cleveland Cavaliers,NBA,5828090
4,2007-08,Cleveland Cavaliers,NBA,13041250
5,2008-09,Cleveland Cavaliers,NBA,14410581
6,2009-10,Cleveland Cavaliers,NBA,15779912
7,2010-11,Miami Heat,NBA,14500000
8,2011-12,Miami Heat,NBA,16022500
9,2012-13,Miami Heat,NBA,17545000


### Repeat with other test URLs to verify

In [8]:
url_path_beg = "https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2F"
url_path_mid = "%2F"
url_path_end = ".html&div=div_all_salaries"

test_id = np.random.choice(np.unique(histor_raptor_player["player_id"]))

test_url  = url_path_beg + test_id[0] + url_path_mid + test_id + url_path_end
test_url

'https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2Fh%2Fhinriki01.html&div=div_all_salaries'

In [9]:
test_id

'hinriki01'

In [10]:
url = test_url
# get HTML data
html = urlopen(url)

# get bs4 obj from HTML
soup = BeautifulSoup(html, features = "lxml")

# get all headers of table as a list
headers = [th.getText() for th in soup.findAll('tr', limit=1)[0].findAll('th')]
print(headers)
# get all headers of table as a list
rows = soup.findAll('tr')[1:]
rows_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]

# change money to numeric
for row in rows_data:
    try:
        row[-1] = int(row[-1].strip('$').replace(',',''))
    except:
        print("\"{}\" couldn't be converted to numeric".format(row[-1]))
        print("replaced with 0")
        row[-1] = 0
df = pd.DataFrame(rows_data)
df.columns = headers[1:]
df[headers[0]] = [[td.getText() for td in rows[i].findAll('th')][0] for i in range(len(rows))]
df = df[headers]

df["player_id"] = [test_id]* df.shape[0]
df

['Season', 'Team', 'Lg', 'Salary']


,Season,Team,Lg,Salary,player_id
0,2003-04,Chicago Bulls,NBA,2185920,hinriki01
1,2004-05,Chicago Bulls,NBA,2349840,hinriki01
2,2005-06,Chicago Bulls,NBA,2513880,hinriki01
3,2006-07,Chicago Bulls,NBA,3192628,hinriki01
4,2007-08,Chicago Bulls,NBA,11250000,hinriki01
5,2008-09,Chicago Bulls,NBA,10000000,hinriki01
6,2009-10,Chicago Bulls,NBA,9500000,hinriki01
7,2010-11,Atlanta Hawks,NBA,9000000,hinriki01
8,2011-12,Atlanta Hawks,NBA,8100000,hinriki01
9,2012-13,Chicago Bulls,NBA,3941000,hinriki01


### Get Salary Data for Every Player

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

all_players = np.unique(histor_raptor_player["player_id"])
len(all_players)

3464

In [6]:
%%time
dfs = []

issues = []
min_salary_issues = []

url_path_beg = "https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fplayers%2F"
url_path_mid = "%2F"
url_path_end = ".html&div=div_all_salaries"
count = 0

for test_id in all_players:
    try:
        test_url  = url_path_beg + test_id[0] + url_path_mid + test_id + url_path_end
        test_url
        url = test_url
#         print("Player:", test_id)
        # get HTML data
        html = urlopen(url)

        # get bs4 obj from HTML
        soup = BeautifulSoup(html, features = "lxml")

        # get all headers of table as a list
        headers = [th.getText() for th in soup.findAll('tr', limit=1)[0].findAll('th')]
#         print(headers)
        # get all headers of table as a list
        rows = soup.findAll('tr')[1:]
        rows_data = [[td.getText() for td in rows[i].findAll('td')]
                            for i in range(len(rows))]

        # change money to numeric
        for row in rows_data:
            try:
                row[-1] = int(row[-1].strip('$').replace(',',''))
            except:
#                 print("\"{}\" couldn't be converted to numeric".format(row[-1]))
#                 print("replaced with 0")
                min_salary_issues.append(test_id)
                row[-1] = 0
        df = pd.DataFrame(rows_data)
        df.columns = headers[1:]
        df[headers[0]] = [[td.getText() for td in rows[i].findAll('th')][0] for i in range(len(rows))]
        df = df[headers]

        df["player_id"] = [test_id]* df.shape[0]
        dfs.append(df)
    except:
        issues.append(test_id)
        print("\t\tThere was an issue with {}".format(test_id))
    count += 1
    if count % 100 == 0:
        print("Finished {} of {}".format(count,len(all_players) ))

		There was an issue with abdulza01
		There was an issue with abernto01
		There was an issue with adamsdo01
		There was an issue with akoonde01
		There was an issue with aleksch01
		There was an issue with allenlu01
		There was an issue with allenra01
		There was an issue with allrela01
		There was an issue with allumda01
		There was an issue with alumape01
		There was an issue with anderan02
		There was an issue with anderdw01
		There was an issue with anderje01
		There was an issue with anderjj01
		There was an issue with anderki01
		There was an issue with andermi01
Finished 100 of 3464
		There was an issue with architi01
		There was an issue with ardji01
		There was an issue with arlaujo01
		There was an issue with armstta01
		There was an issue with austike01
		There was an issue with averibi01
		There was an issue with awtrede01
		There was an issue with baileca01
		There was an issue with bailegu01
		There was an issue with bakerla01
		There was an issue with bakerma01
		There w

		There was an issue with gillihe01
		There was an issue with givenja01
		There was an issue with glennmi01
		There was an issue with gondrgl01
		There was an issue with goodrga01
		There was an issue with graceri01
		There was an issue with grahaor01
		There was an issue with grangst01
		There was an issue with gravebu01
		There was an issue with grayev01
		There was an issue with grayle01
		There was an issue with greenke01
		There was an issue with greenmi01
		There was an issue with greenst01
		There was an issue with greento01
		There was an issue with greigjo01
		There was an issue with griffgr01
		There was an issue with griffpa01
Finished 1200 of 3464
		There was an issue with grossbo01
		There was an issue with grundan01
		There was an issue with hackeru01
		There was an issue with haganas01
		There was an issue with hagangl01
		There was an issue with hamilro01
		There was an issue with hansegl01
		There was an issue with hansela01
		There was an issue with hansore01
		There 

		There was an issue with nicksca01
		There was an issue with nilesmi01
		There was an issue with nixdy01
		There was an issue with normaco01
		There was an issue with norrisy01
		There was an issue with norwowi01
		There was an issue with nuttde01
Finished 2300 of 3464
		There was an issue with oleynfr01
		There was an issue with olivejo01
		There was an issue with owensed01
		There was an issue with owensto01
		There was an issue with pacejo01
		There was an issue with parkeso01
		There was an issue with paspaza01
		There was an issue with patrimy01
		There was an issue with paultbi01
Finished 2400 of 3464
		There was an issue with peckwi01
		There was an issue with pellosa01
		There was an issue with penigde01
		There was an issue with perrycu01
		There was an issue with pheglro01
		There was an issue with phelpde01
		There was an issue with philled01
		There was an issue with pietkst01
		There was an issue with pinknke01
		There was an issue with pinonjo01
		There was an issue with

		There was an issue with youngmi01
		There was an issue with youngpe01
		There was an issue with zenoto01
		There was an issue with zevenph01
		There was an issue with zimmede01
		There was an issue with zoetji01
CPU times: user 46.1 s, sys: 1.91 s, total: 48 s
Wall time: 7min 8s


In [7]:
len(issues)

671

In [8]:
len(min_salary_issues)

248

In [11]:
len(np.unique(min_salary_issues))

207

671 players did not have salary data on Basketball Reference. Of the remaining players, 207 players had less than the minumum tracked value for a salary. This is typically indicative of players on 10-day contracts.

In [14]:
player_salaries = pd.concat(dfs)

In [15]:
player_salaries

,Season,Team,Lg,Salary,player_id
0,1990-91,Portland Trail Blazers,NBA,395000,abdelal01
1,1991-92,Portland Trail Blazers,NBA,494000,abdelal01
2,1992-93,Boston Celtics,NBA,500000,abdelal01
3,1993-94,Boston Celtics,NBA,805000,abdelal01
4,1994-95,Sacramento Kings,NBA,650000,abdelal01
...,...,...,...,...,...
1,2017-18,Los Angeles Lakers,NBA,1312611,zubaciv01
2,2018-19,Los Angeles Clippers,NBA,1544951,zubaciv01
3,2019-20,Los Angeles Clippers,NBA,6481482,zubaciv01
4,2020-21,Los Angeles Clippers,NBA,7000000,zubaciv01


In [25]:
players_no_salary = histor_raptor_player[histor_raptor_player["player_id"].isin(issues)].sort_values(by = ["season"])

In [38]:
players_no_salary

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
33,Zaid Abdul-Aziz,abdulza01,1977,441,195,-1.926415,-0.368835,-2.295249,0.044927,0.044927,0.000000,-1.817927,-1.624327,-3.442253,-0.751147
12566,Frank Oleynick,oleynfr01,1977,1161,516,-4.196373,-2.650225,-6.846597,-1.078375,-1.078375,0.000000,-4.477565,-3.056454,-7.534019,-0.010865
3874,Bob Dandridge,dandrbo01,1977,5708,2501,1.370782,-0.677030,0.693752,4.387709,4.387709,0.000000,1.478683,-0.379360,1.099323,-0.160777
3871,Louie Dampier,dampilo01,1977,3992,1696,-1.332597,-2.504620,-3.837217,-0.938706,-0.875132,-0.063574,-1.667018,-2.630346,-4.297364,-0.540854
12762,Tom Owens,owensto01,1977,1001,469,-1.999711,-1.486433,-3.486144,-0.177824,-0.182056,0.004232,-1.944544,-1.469716,-3.414260,1.109338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4134,Tyler Davis,davisty01,2019,2,1,-40.825921,61.113882,20.287961,0.011423,0.011423,0.000000,-26.935305,42.989063,16.053758,-2.873834
10072,Zach Lofton,loftoza01,2019,9,4,-16.066038,12.480421,-3.585617,-0.001840,-0.001840,0.000000,-8.381958,8.192370,-0.189588,8.076561
13425,Shamorie Ponds,pondssh01,2020,23,11,12.491487,-29.295219,-16.803733,-0.077846,-0.077846,0.000000,8.207066,-23.489402,-15.282336,-1.328036
6654,Ashton Hagans,haganas01,2021,10,4,-18.914996,0.263367,-18.651629,-0.034335,-0.034335,0.000000,-15.697182,-7.847226,-23.544408,5.858741


In [31]:
players_no_salary.describe()

,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
count,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000
mean,1982.020012,2295.469375,1045.477865,-2.237268,-0.753272,-2.990540,0.990450,0.930464,0.059986,-2.292278,-0.989143,-3.281422,0.388081
std,6.520056,2078.516658,939.738354,4.205564,2.847334,5.304511,2.099636,1.968778,0.264761,3.827472,2.626663,5.250698,0.982594
min,1977.000000,2.000000,1.000000,-43.263751,-43.212808,-56.491613,-2.800014,-2.800014,-0.470393,-38.768411,-31.747427,-57.005728,-2.873834
25%,1978.000000,342.000000,158.000000,-3.523539,-1.726361,-4.592770,-0.171967,-0.171967,0.000000,-3.613267,-1.995219,-5.150997,-0.097346
50%,1980.000000,1786.000000,818.000000,-1.658400,-0.750911,-2.310807,0.072576,0.072575,0.000000,-1.862484,-0.928243,-2.667437,0.224527
75%,1983.000000,3835.000000,1750.000000,-0.184964,0.177675,-0.447659,1.488067,1.423694,0.000000,-0.357607,0.095955,-0.591813,0.680869
max,2021.000000,9271.000000,4114.000000,35.634015,61.113882,37.776924,10.724776,10.041522,3.327882,32.053844,42.989063,36.563969,11.219749


In [35]:
players_with_salary = histor_raptor_player[~histor_raptor_player["player_id"].isin(issues)].sort_values(by = ["season"])

In [36]:
players_with_salary.describe()

,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
count,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16904.000000,16903.000000
mean,2003.131981,2818.693150,1405.260116,-1.174853,-0.287070,-1.461923,2.092966,1.890679,0.202287,-1.176340,-0.477281,-1.653621,0.181159
std,11.514536,2025.873879,1006.121955,3.352068,2.272323,4.285366,3.566670,3.165211,0.611427,3.099144,2.179176,4.222285,0.782535
min,1977.000000,1.000000,0.000000,-82.192072,-38.107211,-103.084801,-7.382978,-7.382978,-1.376520,-71.510086,-30.253953,-101.442650,-6.892146
25%,1994.000000,970.750000,488.000000,-2.759704,-1.375401,-3.455340,-0.108642,-0.109430,0.000000,-2.792950,-1.579456,-3.860270,-0.248591
50%,2004.000000,2648.000000,1320.000000,-1.041486,-0.369622,-1.267265,0.680709,0.651414,0.000000,-1.120926,-0.489826,-1.501972,0.085173
75%,2013.000000,4403.250000,2189.000000,0.622922,0.737739,0.833301,3.291680,3.076989,0.093295,0.544120,0.674618,0.794710,0.502756
max,2021.000000,8822.000000,4270.000000,53.228864,62.469205,72.622361,28.762877,24.355978,7.006934,42.890281,41.704602,49.106166,23.732063


In [37]:
players_with_salary[players_with_salary["mp"] <10]

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
3497,Michael Cooper,coopemi01,1979,16,7,-5.462788,3.235257,-2.227531,0.001900,0.001900,0.0,-4.077774,2.382378,-1.695396,1.928041
8801,Charles Jones,jonesch01,1984,6,3,-24.663904,-3.683648,-28.347552,-0.042313,-0.042313,0.0,-21.478926,-1.359169,-22.838095,8.214553
6540,Claude Gregory,gregocl01,1986,4,2,-12.765384,17.215010,4.449626,0.007932,0.007932,0.0,-2.587644,15.673051,13.085407,8.241317
9647,Jeff Lamp,lampje01,1988,15,7,0.290932,-3.592323,-3.301391,-0.001970,-0.001970,0.0,-1.470122,-3.883349,-5.353471,0.047175
16483,Sedric Toney,toneyse01,1989,19,9,-15.827702,-6.164754,-21.992456,-0.092102,-0.092102,0.0,-13.202211,-7.615460,-20.817670,4.318227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15874,Max Strus,strusma01,2020,15,6,18.824392,10.588595,29.412987,0.100705,0.100705,0.0,14.792558,3.614403,18.406961,2.331139
2609,Devontae Cacok,cacokde01,2020,22,9,9.398633,-14.987397,-5.588764,-0.013154,-0.013154,0.0,5.341852,-11.502290,-6.160438,0.930523
7066,Udonis Haslem,hasleud01,2021,7,3,19.361882,28.111728,47.473611,0.077568,0.077568,0.0,18.507221,22.519050,41.026271,0.914219
10373,Will Magnay,magnawi01,2021,7,3,-31.891332,-22.681956,-54.573289,-0.082896,-0.082896,0.0,-20.282359,-18.015490,-38.297848,4.551666


In [46]:
players_no_salary[players_no_salary["mp"]>500]

,player_name,player_id,season,poss,mp,raptor_offense,raptor_defense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_defense,predator_total,pace_impact
12566,Frank Oleynick,oleynfr01,1977,1161,516,-4.196373,-2.650225,-6.846597,-1.078375,-1.078375,0.000000,-4.477565,-3.056454,-7.534019,-0.010865
3874,Bob Dandridge,dandrbo01,1977,5708,2501,1.370782,-0.677030,0.693752,4.387709,4.387709,0.000000,1.478683,-0.379360,1.099323,-0.160777
3871,Louie Dampier,dampilo01,1977,3992,1696,-1.332597,-2.504620,-3.837217,-0.938706,-0.875132,-0.063574,-1.667018,-2.630346,-4.297364,-0.540854
12886,Sonny Parker,parkeso01,1977,2279,1009,1.773610,0.837787,2.611397,2.758639,2.465851,0.292788,1.344103,-0.168530,1.175573,-0.325797
12980,Billy Paultz,paultbi01,1977,6517,2768,-0.916101,-0.418229,-1.334330,1.996910,1.863814,0.133097,-0.502390,-0.418614,-0.921004,-0.438429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,Randy Allen,allenra01,1990,1520,746,-3.464879,-1.076264,-4.541144,-0.684823,-0.684823,0.000000,-4.048901,-1.300064,-5.348965,0.452391
10184,Sidney Lowe,lowesi01,1990,3348,1744,-1.034685,-0.535614,-1.570300,1.044925,1.044925,0.000000,-1.751731,-0.791089,-2.542819,-0.451465
9247,Warren Kidd,kiddwa01,1994,1783,884,-2.107090,-1.477960,-3.585051,-0.378412,-0.378412,0.000000,-2.674652,-1.293625,-3.968277,0.457497
10165,Ryan Lorthridge,lorthry01,1995,1386,672,-1.306732,-1.656392,-2.963124,-0.072841,-0.072841,0.000000,-1.782613,-2.308495,-4.091108,-0.295026


### Other salary data sites
--- 
https://www.spotrac.com/nba/cba/minimum/

In [48]:
player_salaries.to_csv("player_salaries.csv")